# Heston Nandi Options

In [1]:
%load_ext autoreload
%autoreload 2
import finoptions as fo
import numpy as np
import pandas as pd

In [2]:
lamb=-0.5; omega=2.3e-6; alpha=2.9e-6; beta=0.85; gamma=184.25; S=100; K=100; t=252; r=0.05/252

In [3]:
opt = fo.heston_nandi_options.HestonNandiOption(S=S, K=K, t=t, r=r, lamb=lamb,omega=omega,alpha=alpha,beta=beta,gamma=gamma)

In [4]:
opt.call()

8.9920997701416

In [5]:
opt.put()


4.115042220213013

In [6]:
opt.delta()

0.6739533898008037

In [7]:
opt.gamma()

0.022111494128823447

In [8]:
const = 1
phi = 20
for k in range(0,500):
    # Model Parameters:
    gamma = gamma + lamb + 1 / 2
    lamb = -1 / 2
    sigma2 = (omega + alpha) / (1 - beta - alpha * gamma ** 2)
    # Function to be integrated:
    cphi0 = phi * np.array([1j])
    cphi = cphi0 + const
    a = cphi * r
    b = lamb * cphi + cphi * cphi / 2
    # fmt: off
    for i in range(1, t):
        a = a + cphi*r + b*omega - np.log(1-2*alpha*b)/2
        b = cphi*(lamb+gamma) - gamma**2/2 + beta*b + 0.5*(cphi-gamma)**2/(1-2*alpha*b)

In [9]:
anp = np.array([0j]*t)
anp[0] = a[0]

In [10]:
bnp = np.array([0j]*t)
bnp[0] = b[0]

# Garch Functions and Sims